In [1]:
!pip install h2o

  Created wheel for h2o: filename=h2o-3.32.1.3-py2.py3-none-any.whl size=164854343 sha256=c30f0e02d7113fa321def42b0078fc0e7ec6f8451a5b1dae812bf9964f6d576d
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\72\00\18\d1ed0b56eb5efd5e96b48828c07bd131ff8829a6d16fcef39d
Successfully built h2o


In [1]:
import h2o
import pandas as pd

In [3]:

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.291-b10, mixed mode)
  Starting server from C:\Users\Administrator\anaconda3\envs\mlp\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ADMINI~1\AppData\Local\Temp\tmpchlo62pz
  JVM stdout: C:\Users\ADMINI~1\AppData\Local\Temp\tmpchlo62pz\h2o_Administrator_started_from_python.out
  JVM stderr: C:\Users\ADMINI~1\AppData\Local\Temp\tmpchlo62pz\h2o_Administrator_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/London
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,1 month and 25 days
H2O_cluster_name:,H2O_from_python_Administrator_g52e3c
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.427 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"accepting new members, healthy"


In [4]:
data = pd.read_csv('dataset_v2.csv')
# data = data.drop(columns=['org_uuid'])
data

,status,categ_1_hot_Administrative Services,categ_1_hot_Advertising,categ_1_hot_Agriculture and Farming,categ_1_hot_Apps,categ_1_hot_Artificial Intelligence,categ_1_hot_Biotechnology,categ_1_hot_Clothing and Apparel,categ_1_hot_Commerce and Shopping,categ_1_hot_Community and Lifestyle,...,total_funding_1hot_1000,total_funding_1hot_10000,total_funding_1hot_l_10000,num_founder_1hot_3,num_founder_1hot_6,num_founder_1hot_10,have_seed,have_series_a,have_series_b,have_series_c
0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
3,1,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,1,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10724,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
10725,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
10726,1,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
10727,1,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0


In [10]:
!pip install -U scikit-learn

In [5]:
from sklearn.model_selection import train_test_split
data1 = h2o.H2OFrame(data)
train, test = data1.split_frame(ratios=[.8], seed=1234)
# X_train, X_test = train_test_split(data, test_size = 0.3)
# X_train_hex = h2o.H2OFrame(X_train)
# X_test_hex = h2o.H2OFrame(X_test)
# X_names =  ['employee_count_1hot_100', 'employee_count_1hot_1000', 'employee_count_1hot_10000', 'total_funding_1hot_100','total_funding_1hot_1000', 'total_funding_1hot_10000', 'total_funding_1hot_l_10000']

X_names = ['status',
 'categ_1_hot_Administrative Services',
 'categ_1_hot_Advertising',
 'categ_1_hot_Agriculture and Farming',
 'categ_1_hot_Apps',
 'categ_1_hot_Artificial Intelligence',
 'categ_1_hot_Biotechnology',
 'categ_1_hot_Clothing and Apparel',
 'categ_1_hot_Commerce and Shopping',
 'categ_1_hot_Community and Lifestyle',
 'categ_1_hot_Consumer Electronics',
 'categ_1_hot_Consumer Goods',
 'categ_1_hot_Content and Publishing',
 'categ_1_hot_Data and Analytics',
 'categ_1_hot_Design',
 'categ_1_hot_Education',
 'categ_1_hot_Energy',
 'categ_1_hot_Events',
 'categ_1_hot_Financial Services',
 'categ_1_hot_Food and Beverage',
 'categ_1_hot_Gaming',
 'categ_1_hot_Government and Military',
 'categ_1_hot_Hardware',
 'categ_1_hot_Health Care',
 'categ_1_hot_Information Technology',
 'categ_1_hot_Internet Services',
 'categ_1_hot_Lending and Investments',
 'categ_1_hot_Manufacturing',
 'categ_1_hot_Media and Entertainment',
 'categ_1_hot_Messaging and Telecommunications',
 'categ_1_hot_Mobile',
 'categ_1_hot_Music and Audio',
 'categ_1_hot_Natural Resources',
 'categ_1_hot_Navigation and Mapping',
 'categ_1_hot_Other',
 'categ_1_hot_Payments',
 'categ_1_hot_Platforms',
 'categ_1_hot_Privacy and Security',
 'categ_1_hot_Professional Services',
 'categ_1_hot_Real Estate',
 'categ_1_hot_Sales and Marketing',
 'categ_1_hot_Science and Engineering',
 'categ_1_hot_Software',
 'categ_1_hot_Sports',
 'categ_1_hot_Sustainability',
 'categ_1_hot_Transportation',
 'categ_1_hot_Travel and Tourism',
 'categ_1_hot_Video',
 'total_funding_1hot_100',
 'total_funding_1hot_1000',
 'total_funding_1hot_10000',
 'total_funding_1hot_l_10000',
 'num_founder_1hot_3',
 'num_founder_1hot_6',
 'num_founder_1hot_10',
 'have_seed',
 'have_series_a',
 'have_series_b',
 'have_series_c']

X_test = test.drop('status').as_data_frame()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
X_train_hex= train.asfactor()
X_test_hex = test.asfactor()
X_train_hex.describe()

Rows:8629
Cols:59




,status,categ_1_hot_Administrative Services,categ_1_hot_Advertising,categ_1_hot_Agriculture and Farming,categ_1_hot_Apps,categ_1_hot_Artificial Intelligence,categ_1_hot_Biotechnology,categ_1_hot_Clothing and Apparel,categ_1_hot_Commerce and Shopping,categ_1_hot_Community and Lifestyle,categ_1_hot_Consumer Electronics,categ_1_hot_Consumer Goods,categ_1_hot_Content and Publishing,categ_1_hot_Data and Analytics,categ_1_hot_Design,categ_1_hot_Education,categ_1_hot_Energy,categ_1_hot_Events,categ_1_hot_Financial Services,categ_1_hot_Food and Beverage,categ_1_hot_Gaming,categ_1_hot_Government and Military,categ_1_hot_Hardware,categ_1_hot_Health Care,categ_1_hot_Information Technology,categ_1_hot_Internet Services,categ_1_hot_Lending and Investments,categ_1_hot_Manufacturing,categ_1_hot_Media and Entertainment,categ_1_hot_Messaging and Telecommunications,categ_1_hot_Mobile,categ_1_hot_Music and Audio,categ_1_hot_Natural Resources,categ_1_hot_Navigation and Mapping,categ_1_hot_Other,categ_1_hot_Payments,categ_1_hot_Platforms,categ_1_hot_Privacy and Security,categ_1_hot_Professional Services,categ_1_hot_Real Estate,categ_1_hot_Sales and Marketing,categ_1_hot_Science and Engineering,categ_1_hot_Software,categ_1_hot_Sports,categ_1_hot_Sustainability,categ_1_hot_Transportation,categ_1_hot_Travel and Tourism,categ_1_hot_Video,total_funding_1hot_100,total_funding_1hot_1000,total_funding_1hot_10000,total_funding_1hot_l_10000,num_founder_1hot_3,num_founder_1hot_6,num_founder_1hot_10,have_seed,have_series_a,have_series_b,have_series_c
type,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum
mins,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
mean,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
maxs,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
sigma,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
zeros,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0
1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0


In [7]:
# Define model
# h2o_rf = H2ORandomForestEstimator(ntrees=200, max_depth=20, nfolds=10)
# Train model
from h2o.automl import H2OAutoML
aml = H2OAutoML(
    max_runtime_secs=(60 * 10),  # 8 hours
    max_models=None,  # no limit
    seed=20
)
aml.train(x=X_names, y='status', training_frame=X_train_hex)


AutoML progress: |
10:51:50.772: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%


In [34]:
import os
lb = aml.leaderboard

# model_ids = list(lb['model_id'].as_data_frame().iloc[:,0])
# out_path = "."

# for m_id in model_ids:
#     mdl = h2o.get_model(m_id)
#     h2o.save_model(model=mdl, path=out_path, force=True)

# h2o.export_file(lb, os.path.join(out_path, 'aml_leaderboard2.h2o'), force=True)
df = lb.head(rows=lb.nrows) 

In [35]:
df

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_AutoML_20210715_105150,0.627242,0.244464,0.956651,0.5,0.252041,0.0635246
GLM_1_AutoML_20210715_105150,0.62441,0.24484,0.95679,0.5,0.25215,0.0635799
GBM_grid__1_AutoML_20210715_105150_model_1,0.62408,0.247025,0.955722,0.5,0.253125,0.0640723
GBM_grid__1_AutoML_20210715_105150_model_12,0.622804,0.246791,0.955868,0.5,0.252953,0.063985
StackedEnsemble_AllModels_AutoML_20210715_105150,0.622408,0.244973,0.955959,0.5,0.252173,0.0635912
GBM_grid__1_AutoML_20210715_105150_model_13,0.621493,0.248838,0.954248,0.5,0.253519,0.0642719
XRT_1_AutoML_20210715_105150,0.621305,0.248698,0.956423,0.5,0.252947,0.0639823
GBM_5_AutoML_20210715_105150,0.620799,0.245998,0.955435,0.5,0.25248,0.0637462
GBM_grid__1_AutoML_20210715_105150_model_11,0.619254,0.245468,0.955545,0.499161,0.252327,0.063669
GBM_grid__1_AutoML_20210715_105150_model_15,0.619172,0.246453,0.954883,0.5,0.252684,0.063849


In [17]:
model = h2o.get_model('GBM_grid__1_AutoML_20210715_105150_model_1')
perf = model.model_performance(X_test_hex)
# model.model_performance()
perf


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.06134264141048438
RMSE: 0.24767446660987155
LogLoss: 0.23658833575959692
Mean Per-Class Error: 0.3756543974407419
AUC: 0.6402859354535749
AUCPR: 0.9621229976348808
Gini: 0.28057187090714986

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6885243129584632: 


,,0,1,Error,Rate
0,0,0.0,139.0,1.0,(139.0/139.0)
1,1,0.0,1961.0,0.0,(0.0/1961.0)
2,Total,0.0,2100.0,0.0662,(139.0/2100.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.688524,0.965772,399.0
1,max f2,0.688524,0.986022,399.0
2,max f0point5,0.760955,0.946482,393.0
3,max accuracy,0.688524,0.933810,399.0
4,max precision,0.992503,1.000000,0.0
5,max recall,0.688524,1.000000,399.0
6,max specificity,0.992503,1.000000,0.0
7,max absolute_mcc,0.945049,0.125103,125.0
8,max min_per_class_accuracy,0.926523,0.585926,182.0
9,max mean_per_class_accuracy,0.931540,0.624346,167.0



Gains/Lift Table: Avg response rate: 93.38 %, avg score: 92.95 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.01,0.989216,1.070882,1.070882,1.000000,0.990506,1.000000,0.990506,0.010709,0.010709,7.088220,7.088220,0.010709
1,2,0.02,0.987398,1.070882,1.070882,1.000000,0.988345,1.000000,0.989425,0.010709,0.021418,7.088220,7.088220,0.021418
2,3,0.03,0.986336,1.019888,1.053884,0.952381,0.986905,0.984127,0.988585,0.010199,0.031617,1.988781,5.388407,0.024422
3,4,0.04,0.985562,1.070882,1.058134,1.000000,0.985868,0.988095,0.987906,0.010709,0.042325,7.088220,5.813361,0.035131
4,5,0.05,0.984467,1.019888,1.050484,0.952381,0.984997,0.980952,0.987324,0.010199,0.052524,1.988781,5.048445,0.038136
5,6,0.10,0.978584,1.060683,1.055584,0.990476,0.981921,0.985714,0.984623,0.053034,0.105558,6.068332,5.558389,0.083976
6,7,0.15,0.973054,1.050484,1.053884,0.980952,0.975965,0.984127,0.981737,0.052524,0.158083,5.048445,5.388407,0.122111
7,8,0.20,0.965838,1.050484,1.053034,0.980952,0.969610,0.983333,0.978705,0.052524,0.210607,5.048445,5.303417,0.160247
8,9,0.30,0.953612,1.024987,1.043685,0.957143,0.959021,0.974603,0.972144,0.102499,0.313106,2.498725,4.368519,0.197998
9,10,0.40,0.942711,1.019888,1.037736,0.952381,0.948066,0.969048,0.966124,0.101989,0.415094,1.988781,3.773585,0.228044


In [11]:
model = h2o.get_model('DRF_1_AutoML_20210715_105150')
# perf = model.model_performance(X_test_hex)
model.model_performance()


ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.06557889248033012
RMSE: 0.25608376067281213
LogLoss: 0.27430738322607606
Mean Per-Class Error: 0.42157549353881685
AUC: 0.5984002023532125
AUCPR: 0.9525863266546644
Gini: 0.1968004047064249

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4804755846659342: 


,,0,1,Error,Rate
0,0,1.0,595.0,0.9983,(595.0/596.0)
1,1,0.0,8033.0,0.0,(0.0/8033.0)
2,Total,1.0,8628.0,0.069,(595.0/8629.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.480476,0.964288,398.0
1,max f2,0.480476,0.985402,398.0
2,max f0point5,0.480476,0.944059,398.0
3,max accuracy,0.480476,0.931046,398.0
4,max precision,0.998359,0.982507,3.0
5,max recall,0.480476,1.000000,398.0
6,max specificity,1.000000,0.991611,0.0
7,max absolute_mcc,0.951004,0.080783,99.0
8,max min_per_class_accuracy,0.933521,0.562305,135.0
9,max mean_per_class_accuracy,0.951004,0.578425,99.0



Gains/Lift Table: Avg response rate: 93.09 %, avg score: 92.91 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.030942,1.000000,1.054078,1.054078,0.981273,1.000000,0.981273,1.000000,0.032615,0.032615,5.407796,5.407796,0.024226
1,2,0.040097,0.998050,1.060597,1.055566,0.987342,0.998678,0.982659,0.999698,0.009710,0.042325,6.059656,5.556631,0.032258
2,3,0.050180,0.997024,1.037153,1.051867,0.965517,0.997521,0.979215,0.999261,0.010457,0.052782,3.715278,5.186659,0.037682
3,4,0.100012,0.991765,1.044216,1.048055,0.972093,0.994438,0.975666,0.996858,0.052035,0.104818,4.421644,4.805482,0.069583
4,5,0.150075,0.985249,1.034409,1.043503,0.962963,0.988555,0.971429,0.994088,0.051786,0.156604,3.440899,4.350269,0.094523
5,6,0.200023,0.978410,1.031824,1.040586,0.960557,0.982019,0.968714,0.991074,0.051537,0.208141,3.182435,4.058649,0.117537
6,7,0.300035,0.965411,1.016937,1.032703,0.946698,0.971883,0.961375,0.984677,0.101705,0.309847,1.693680,3.270326,0.142062
7,8,0.400046,0.952166,1.008224,1.026583,0.938586,0.958708,0.955678,0.978185,0.100834,0.410681,0.822375,2.658338,0.153970
8,9,0.500058,0.939850,0.995777,1.020422,0.926999,0.945808,0.949942,0.971709,0.099589,0.510270,-0.422345,2.042202,0.147854
9,10,0.599954,0.927406,0.985716,1.014643,0.917633,0.933615,0.944562,0.965366,0.098469,0.608739,-1.428374,1.464331,0.127195


In [12]:
model = h2o.get_model('DeepLearning_1_AutoML_20210715_105150')
# perf = model.model_performance(X_test_hex)
model.model_performance()


ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.07755676675178935
RMSE: 0.2784901555742848
LogLoss: 0.3000466973600749
Mean Per-Class Error: 0.3967426312768554
AUC: 0.6385034425945992
AUCPR: 0.9590462396821524
Gini: 0.27700688518919847

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6421055610570181: 


,,0,1,Error,Rate
0,0,0.0,596.0,1.0,(596.0/596.0)
1,1,0.0,8033.0,0.0,(0.0/8033.0)
2,Total,0.0,8629.0,0.0691,(596.0/8629.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.642106,0.964230,399.0
1,max f2,0.642106,0.985378,399.0
2,max f0point5,0.673029,0.944093,393.0
3,max accuracy,0.650834,0.930931,398.0
4,max precision,0.975334,1.000000,0.0
5,max recall,0.642106,1.000000,399.0
6,max specificity,0.975334,1.000000,0.0
7,max absolute_mcc,0.775494,0.110888,277.0
8,max min_per_class_accuracy,0.795998,0.592282,239.0
9,max mean_per_class_accuracy,0.785277,0.603257,259.0



Gains/Lift Table: Avg response rate: 93.09 %, avg score: 81.53 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010082,0.952548,1.061847,1.061847,0.988506,0.959110,0.988506,0.959110,0.010706,0.010706,6.184689,6.184689,0.009028
1,2,0.020049,0.942285,1.061703,1.061776,0.988372,0.946734,0.988439,0.952958,0.010581,0.021287,6.170332,6.177552,0.017931
2,3,0.030015,0.936173,1.061703,1.061752,0.988372,0.939977,0.988417,0.948648,0.010581,0.031869,6.170332,6.175155,0.026835
3,4,0.040097,0.929962,1.049500,1.058671,0.977011,0.932843,0.985549,0.944674,0.010581,0.042450,4.949984,5.867092,0.034061
4,5,0.050064,0.923785,1.061703,1.059275,0.988372,0.926734,0.986111,0.941103,0.010581,0.053031,6.170332,5.927459,0.042964
5,6,0.100012,0.900621,1.056748,1.058013,0.983759,0.912040,0.984936,0.926588,0.052782,0.105814,5.674764,5.801258,0.084001
6,7,0.150075,0.881964,1.036896,1.050968,0.965278,0.890936,0.978378,0.914695,0.051911,0.157724,3.689555,5.096813,0.110745
7,8,0.200023,0.867952,1.034317,1.046810,0.962877,0.874651,0.974508,0.904696,0.051662,0.209386,3.431668,4.681009,0.135561
8,9,0.300035,0.846716,1.030629,1.041416,0.959444,0.856551,0.969486,0.888647,0.103075,0.312461,3.062873,4.141631,0.179911
9,10,0.400046,0.828625,0.997021,1.030318,0.928158,0.837695,0.959154,0.875909,0.099714,0.412175,-0.297873,3.031755,0.175598


In [13]:
model = h2o.get_model('GLM_1_AutoML_20210715_105150')
# perf = model.model_performance(X_test_hex)
model.model_performance()


ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.06314625649140404
RMSE: 0.25128918896642577
LogLoss: 0.24171345286601462
Null degrees of freedom: 8628
Residual degrees of freedom: 8514
Null deviance: 4335.643455378077
Residual deviance: 4171.49076956168
AIC: 4401.49076956168
AUC: 0.6570337583976166
AUCPR: 0.9614293349883738
Gini: 0.31406751679523315

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.8395718159346054: 


,,0,1,Error,Rate
0,0,0.0,596.0,1.0,(596.0/596.0)
1,1,0.0,8033.0,0.0,(0.0/8033.0)
2,Total,0.0,8629.0,0.0691,(596.0/8629.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.839572,0.964230,399.0
1,max f2,0.839572,0.985378,399.0
2,max f0point5,0.850332,0.944002,396.0
3,max accuracy,0.839572,0.930931,399.0
4,max precision,0.987140,1.000000,0.0
5,max recall,0.839572,1.000000,399.0
6,max specificity,0.987140,1.000000,0.0
7,max absolute_mcc,0.921979,0.123242,206.0
8,max min_per_class_accuracy,0.921790,0.615212,207.0
9,max mean_per_class_accuracy,0.922247,0.619370,205.0



Gains/Lift Table: Avg response rate: 93.09 %, avg score: 93.09 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010546,0.980417,1.050585,1.050585,0.978022,0.982086,0.978022,0.982086,0.011079,0.011079,5.058529,5.058529,0.007724
1,2,0.020049,0.979302,1.061094,1.055566,0.987805,0.979791,0.982659,0.980998,0.010083,0.021163,6.109402,5.556631,0.016129
2,3,0.030015,0.976101,1.074194,1.061752,1.000000,0.977463,0.988417,0.979824,0.010706,0.031869,7.419395,6.175155,0.026835
3,4,0.040097,0.974295,1.074194,1.064880,1.000000,0.975209,0.991329,0.978664,0.010830,0.042699,7.419395,6.488013,0.037665
4,5,0.050064,0.972883,1.036722,1.059275,0.965116,0.973685,0.986111,0.977673,0.010332,0.053031,3.672207,5.927459,0.042964
5,6,0.100012,0.967524,1.049271,1.054278,0.976798,0.969892,0.981460,0.973787,0.052409,0.105440,4.927066,5.427842,0.078594
6,7,0.150075,0.962859,1.061761,1.056775,0.988426,0.965277,0.983784,0.970948,0.053156,0.158596,6.176115,5.677459,0.123361
7,8,0.200023,0.958641,1.041794,1.053034,0.969838,0.960889,0.980301,0.968436,0.052035,0.210631,4.179367,5.303370,0.153584
8,9,0.300498,0.949890,1.030830,1.045610,0.959631,0.954084,0.973390,0.963638,0.103573,0.314204,3.082972,4.560954,0.198432
9,10,0.400046,0.940783,1.007917,1.036230,0.938300,0.945184,0.964658,0.959045,0.100336,0.414540,0.791656,3.622997,0.209842


In [59]:
from h2o.estimators import H2OGradientBoostingEstimator
pros_gbm = H2OGradientBoostingEstimator(nfolds=5,
                                        seed=1234,
                                        keep_cross_validation_predictions = True)
pros_gbm.train(x=X_names, y='status', training_frame=X_train_hex)

# Eval performance:
perf = pros_gbm.model_performance(X_test_hex)
# Generate predictions on a test set (if necessary):
# pred = pros_gbm.predict()

# Extract feature interactions:
# feature_interactions = pros_gbm.feature_interaction()

C:\Users\Administrator\anaconda3\envs\mlp\lib\site-packages\h2o\estimators\estimator_base.py:200: RuntimeWarning: Dropping bad and constant columns: [have_series_c]
  warnings.warn(mesg["message"], RuntimeWarning)


gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [75]:
pred = pros_gbm.predict(X_test_hex)
# pred['predict'].unique()

# print(train.loc[train['status']==0].shape)

gbm prediction progress: |████████████████████████████████████████████████| 100%


C1
0
1


In [28]:
class H2OProbWrapper:
    def __init__(self, h2o_model, feature_names):
        self.h2o_model = h2o_model
        self.feature_names = feature_names
    def predict_binary_prob(self, X):
        if isinstance(X, pd.Series):
            X = X.values.reshape(1,-1)
        self.dataframe= pd.DataFrame(X, columns=self.feature_names)
        self.predictions = self.h2o_model.predict(h2o.H2OFrame(self.dataframe)).as_data_frame().values
        return self.predictions.astype('float64')[:,-1] 

In [29]:
h2o_wrapper = H2OProbWrapper(model,X_names)

In [21]:
!pip install shap

In [33]:
import shap
explainer = shap.KernelExplainer(h2o_wrapper.predict_binary_prob, X_test_hex)
xgboost_shap_values = explainer.shap_values(X_test)

AssertionError: Unknown type passed as data object: <class 'h2o.frame.H2OFrame'>

In [ ]:
shap.summary_plot(xgboost_shap_values, X_test)